In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
%matplotlib inline

In [ ]:
sns.set()

In [ ]:
data = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv',index_col='enrollee_id')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data['relevent_experience'].value_counts()

In [ ]:
data['gender'].value_counts()

In [ ]:
data['gender'].fillna('Male',inplace=True)

In [ ]:
data['enrolled_university'].value_counts()

In [ ]:
data['enrolled_university'].fillna('no_enrollment',inplace=True)

In [ ]:
data['education_level'].value_counts()

In [ ]:
data[data['education_level'].isnull()]

In [ ]:
sns.countplot(data['education_level'],hue=data['relevent_experience'])

In [ ]:
sns.scatterplot(data['city_development_index'],data['experience'],hue=data['education_level'])
plt.legend(bbox_to_anchor=(1, 1),loc='upper left')

In [ ]:
a1=data[data['relevent_experience']=='No relevent experience']['education_level'].fillna('High School')
a2=data[data['relevent_experience']=='Has relevent experience']['education_level'].fillna('Graduate')
a=pd.concat([a1,a2])
data['education_level']=a

In [ ]:
data['major_discipline'].value_counts()

In [ ]:
data['major_discipline'].fillna('STEM',inplace=True)

In [ ]:
sns.histplot(data['experience'].replace({'>20':'21','<1':'0'}).astype(float),kde=True)

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data['experience'].replace({'>20':'21','<1':'0'}).astype(float),hue=data['education_level'])

In [ ]:
data['experience'].fillna('>20',inplace=True)

In [ ]:
data['company_type'].value_counts()

In [ ]:
data['company_type'].fillna('Pvt Ltd',inplace=True)

In [ ]:
data['last_new_job'].value_counts()

In [ ]:
data['last_new_job'].fillna('1',inplace=True)

In [ ]:
data['company_size'].value_counts()

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data['company_size'],data['experience'].replace({'>20':'21','<1':'0'}).astype(float))

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data['company_size'],data['city_development_index'])

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data['company_size'],hue=data['company_type'])

In [ ]:
#No specific way to find the missing company size. hence have to drop NA values
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
target=pd.DataFrame(data['target'].astype(int))
required_data=data.drop(['city','target'],axis=1)

In [ ]:
target.head()

In [ ]:
required_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
def toLabelEncoder(data):
    LE=LabelEncoder()
    return LE.fit_transform(data)

In [ ]:
required_data=required_data.apply(toLabelEncoder)

In [ ]:
required_data['city_development_index']=data['city_development_index']

In [ ]:
required_data.head()

In [ ]:
SC=StandardScaler()

In [ ]:
required_data_sc=pd.DataFrame(SC.fit_transform(required_data),index=required_data.index,columns=required_data.columns)

In [ ]:
required_data_sc.head()

# Random Forest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(required_data, target, test_size=0.3, random_state=25)

In [ ]:
result=pd.DataFrame(y_test)

In [ ]:
RFC=RandomForestClassifier()
RFC.fit(X_train,y_train)

In [ ]:
result['RFC']=RFC.predict(X_test)

In [ ]:
print(classification_report(result['target'],result['RFC']))

In [ ]:
print(confusion_matrix(result['target'],result['RFC']))

In [ ]:
X_train_sc, X_test_sc, y_train, y_test = train_test_split(required_data_sc, target, test_size=0.3, random_state=25)

In [ ]:
RFC=RandomForestClassifier()
RFC.fit(X_train_sc,y_train)

In [ ]:
result['RFC_sc']=RFC.predict(X_test_sc)

In [ ]:
print(classification_report(result['target'],result['RFC_sc']))

In [ ]:
print(confusion_matrix(result['target'],result['RFC_sc']))

# kNeighbour

In [ ]:
score=list()
for i in range(1,20):
    KNC=KNeighborsClassifier(n_neighbors=i)
    KNC.fit(X_train,y_train)
    KNC.predict(X_test)
    score.append(KNC.score(X_test,y_test))
score=np.array(score)

In [ ]:
score=pd.DataFrame(score)
plt.figure(figsize=(15,5))
sns.lineplot(score.index,score[0])
plt.xticks(range(1,20))

In [ ]:
KNC=KNeighborsClassifier(n_neighbors=5)
KNC.fit(X_train,y_train)
result['KNC']=KNC.predict(X_test)

In [ ]:
print(classification_report(result['target'],result['KNC']))

In [ ]:
print(confusion_matrix(result['target'],result['KNC']))

In [ ]:
score=list()
for i in range(1,20):
    KNC=KNeighborsClassifier(n_neighbors=i)
    KNC.fit(X_train_sc,y_train)
    KNC.predict(X_test_sc)
    score.append(KNC.score(X_test_sc,y_test))
score=np.array(score)

In [ ]:
score=pd.DataFrame(score)
plt.figure(figsize=(15,5))
sns.lineplot(score.index,score[0])
plt.xticks(range(1,20))

In [ ]:
KNC=KNeighborsClassifier(n_neighbors=11)
KNC.fit(X_train_sc,y_train)
result['KNC_sc']=KNC.predict(X_test_sc)

In [ ]:
print(classification_report(result['target'],result['KNC_sc']))

In [ ]:
print(confusion_matrix(result['target'],result['KNC_sc']))

In [ ]:
result.head(15)